##Text cleaning and preprocessing

In [1]:
!python -m spacy download en
import pandas as pd
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from time import time 
from collections import defaultdict 
import spacy
from google.colab import files

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


a) Eliminar Stop-Words

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
def eliminar_StopWords(frase):
  word_tokens = word_tokenize(frase)
  filtered_sentence = [w for w in word_tokens if not w in stop_words]
  filtered_sentence = []
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)
  return filtered_sentence

In [4]:
def stop_Words(lista_de_strings):
  
  elementos_sin_stop_words = []

  for frase in lista_de_strings:
    elementos_sin_stop_words.append(eliminar_StopWords(frase))
  
  train_sin_SW = []

  for elemento in elementos_sin_stop_words:
    string = ' '.join(elemento)
    train_sin_SW.append(string)

  return train_sin_SW

b) Reducción de ruido

In [5]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},
        {r'\s+': u' '},
        {r'\s*<br\s*/?>\s*': u'\n'},
        {r'</(div)\s*>\s*': u'\n'},  
        {r'</(p|h\d)\s*>\s*': u'\n\n'},
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},
        {r'[ \t]*<[^<]*?/?>': u''},
        {r'^\s+': u''},
        {r'https?://\S+|www\.\S+': u''},
        {r'<.*?>': u''}
    ]
    for rule in rules:
      for (k, v) in rule.items():
          regex = re.compile(k)
          text = regex.sub(v, text)
    text = text.rstrip()
    return text.lower()



In [6]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [7]:
def unslang(text):
  slang_abbrev_dict = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'B4N': 'Bye For Now',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you',
    'ILU': 'I Love You',
    'IMHO': 'In My Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My Ass Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'OMG': 'Oh My God',
    'PITA': 'Pain In The Ass',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My Ass Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The Fuck',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait',
    '7K': 'Sick:-D Laugher'
}
  words = text.split(" ")
  unslanged_text = ''
  for word in words:
    if word.upper() in slang_abbrev_dict.keys():
      unslanged_text += slang_abbrev_dict[word.upper()]
    else:
      unslanged_text += word
    return unslanged_text

In [16]:
def reduccion_de_ruido(lista_de_strings):

  sin_ruido = []

  for text in lista_de_strings:
    final = text_cleaner(text)
    final = remove_emoji(final)
    final = unslang(final)
    sin_ruido.append(text)

  return sin_ruido

c) Lemmatizamos (elimina prefijos y sufijos redundantes de las palabras).

In [9]:
nlp = spacy.load('en')

def cleaning(doc):

    txt = [token.lemma_ for token in doc if not token.is_stop]

    if len(txt) > 2:
        return ' '.join(txt)

In [10]:
def lemmatizacion(lista_de_strings):
  limpio = [cleaning(tweet) for tweet in nlp.pipe(lista_de_strings, batch_size=5000, n_threads=-1)]
  return limpio

Guardamos la limpieza en un dataframe para luego utilizarlo en las predicciones

In [11]:
def genenerado_guardado_df(lista_limpia):
  limpio_df = pd.DataFrame({'text':lista_limpia})
  limpio_df = limpio_df.astype({'text': 'string'})
  limpio_df = limpio_df.fillna('')
  limpio_df.to_csv('dataframe_limpio.csv', index=False)
  files.download('dataframe_limpio.csv')

In [12]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [13]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [19]:
#sw = stop_Words(train.text)
sw = stop_Words(test.text)
rd = reduccion_de_ruido(sw)
lm = lemmatizacion(rd)
genenerado_guardado_df(lm)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>